The purpose of this notebook is to find all of the possible decay chains based off of the data in livechart. 

In [47]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Downloads
#plotlyjs()
data_dir = ("C:\\Cross-Section-Data\\")
cross_section_dir = data_dir
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"
endf8_decay_dir = cross_section_dir * "ENDF_Libraries-2\\ENDF-B-VIII.0\\decay\\"
jeff33_decay_dir = cross_section_dir * "ENDF_Libraries-2\\JEFF-3.3\\decay\\"
jendl5_decay_dir = cross_section_dir * "jendl5-dec_upd5\\"

"C:\\Cross-Section-Data\\jendl5-dec_upd5\\"

Now I will find the average decay energies for each nuclide. For the datasets I am using, the average decay energies are all given in keV. 

In [48]:
function get_mass_name(nuclide)
    index = 1
    while (tryparse(Int64, string(nuclide[index])) != nothing && index <= length(nuclide))
        index += 1
    end
    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"];

Rename files in directory to be easier to parse by removing zero-padding.

In [49]:
function unzero_pad(num_str) 
    if (num_str == '0')
        return num_str
    end
    i = 1
    while i < length(num_str) && num_str[i] == '0'
        i += 1
    end
    return num_str[i : end]
end
function rename_files_in_dir(dir, cutoff_index)
    for f in readdir(dir)
        if tryparse(Int64, string(f[1])) == nothing
            mv(dir * f, dir * f[cutoff_index:end])
        end
        if f != join(unzero_pad.(split(f, "-")), "-")
            mv(dir * f, dir * join(unzero_pad.(split(f, "-")), "-"))
        end
    end
end
rename_files_in_dir(endf8_decay_dir, 12)
rename_files_in_dir(jeff33_decay_dir, 12)
rename_files_in_dir(jendl5_decay_dir, 4)

In [50]:
function get_decay_energy_from_file(file_as_arr, search_string)
    line = [l for l in file_as_arr if length(l) > length(search_string)
             && occursin(search_string, l)]
    if (length(line) > 0)
        line = String(line[1])
        line = join(split(line, search_string, keepempty = false))
        return tryparse(Float64, join(split(line, " ", keepempty = false)[1]))
    else 
        return "Energy not found"
    end
end

get_decay_energy_from_file (generic function with 1 method)

This dictionary stores what strings to look to for in the datasets to find the average energy for each decay type, and for each directory. 

In [51]:
dir_search_string_dict = Dict([
    jeff33_decay_dir => Dict(["alpha" => "ALPHA ENERGY                   =",
    "beta-minus" => "MEAN BETA- ENERGY              =","gamma" => "MEAN GAMMA ENERGY              =", 
    "x-ray" => "MEAN X-RAY ENERGY              =", "auger" => "MEAN AUGER ELECTRON ENERGY     ="
    ]),

    endf8_decay_dir => Dict(["alpha" => "Mean Alpha Energy:", 
    "beta-minus" => "Mean B- Energy:", "beta-plus" => "Mean B+ Energy:",
    "gamma" => "Mean Gamma Energy:", "auger" => "Mean CE+Auger Energy:",
    "neutron" => "Mean Neutron Energy:", "proton" => "Mean Neutron Energy:"]),

    jendl5_decay_dir => Dict(["alpha" => "Mean Alpha Energy",
    "beta-minus" => "Mean B- Energy", "gamma" => "Mean Gamma Energy",
    "neutron" => "Mean Neutron Energy", "proton" => "Mean Proton Energy",
    "auger" => "Mean CE+Auger Energy", 
    "beta-plus and electron capture" => "Mean B+/EC Energy"])  
])

Dict{String, Dict{String, String}} with 3 entries:
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy", "beta-plus…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy:", "proton"=…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"MEAN AUGER ELECTRON ENERGY     ="…

In [52]:
function find_avg_decay_energy(nuclide, decay_type, dir)
    a, el = get_mass_name(nuclide)
    el = string(el)
    z = [i for i in 1:length(element_symbols) if element_symbols[i] == el][1]
    file_name = string(z) * "-" * el * "-" * string(a) * ".dat"
    file_path = dir * file_name
    file_as_array = split(open(f->read(f, String), file_path), "\n")
    search_string = dir_search_string_dict[dir][decay_type]
    return get_decay_energy_from_file(file_as_array, search_string)
end
dir_list = keys(dir_search_string_dict)
function find_avg_decay_energy(nuclide, decay_type)
    for dir in dir_list
        energy = find_avg_decay_energy(nuclide, decay_type, dir)
        if energy != "Energy not found"
            return energy
        end
    end
    return "Energy not found"
end

find_avg_decay_energy (generic function with 2 methods)

Now for some examples. Again, all average decay energies are given in keV. 

In [53]:
out = find_avg_decay_energy("238U", "alpha", jeff33_decay_dir)

4187.0737

In [54]:
out = find_avg_decay_energy("3H", "beta-minus", jendl5_decay_dir)

5.69

In [55]:
avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jendl5_decay_dir)

"Energy not found"

In [56]:
avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jendl5_decay_dir)
if avg_decay_energy == "Energy not found"
    avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jeff33_decay_dir)
end

45.95

In [57]:
find_avg_decay_energy("238U", "alpha", endf8_decay_dir)

4187.0

In [58]:
live_chart_file_name = "all_livechart_decays_and_emissions.csv"
all_live_chart_data = CSV.read(parent_dir * "ImportedData\\" * live_chart_file_name,
                        DataFrame, stringtype = String)[:,2:end]

,z,n,symbol,radius,unc_r,abundance,abundance_unc,energy_shift
,Int64,Int64,String,String,Float64?,String,String?,String?
1,0,1,Nn,-0.1149,0.0027,,missing,
2,0,4,N,,missing,,missing,
3,0,6,N,,missing,,missing,
4,1,0,H,0.8783,0.0086,99.9855,78,
5,1,1,H,2.1421,0.0088,0.0145,78,
6,1,2,H,1.7591,0.0363,,missing,
7,1,3,H,,missing,,missing,
8,1,4,H,,missing,,missing,
9,1,5,H,,missing,,missing,


In [59]:
all_live_chart_data[!, "nuclide"] = [string(all_live_chart_data[row, "z"] + 
                        all_live_chart_data[row, "n"]) * all_live_chart_data[row, "symbol"]
                        for row in 1:size(all_live_chart_data)[1]]
livechart_nuclides = unique(all_live_chart_data[!, "nuclide"])

3367-element Vector{String}:
 "1Nn"
 "4N"
 "6N"
 "1H"
 "2H"
 "3H"
 "4H"
 "5H"
 "6H"
 "7H"
 "3He"
 "4He"
 "5He"
 ⋮
 "287Mc"
 "288Mc"
 "289Mc"
 "290Mc"
 "290Lv"
 "291Lv"
 "292Lv"
 "293Lv"
 "294Lv"
 "293Ts"
 "294Ts"
 "294Og"

In [60]:
top_decay_probs = [tryparse(BigFloat, p) for p in all_live_chart_data[!,"decay_1_%"] 
                    if tryparse(Float64, p) != nothing]
decay_prob_df = DataFrame(Prob = top_decay_probs)

,Prob
,BigFloat
1,100.0
2,100.0
3,100.0
4,100.0
5,100.0
6,100.0
7,100.0
8,100.0
9,100.0


In [61]:
plot(decay_prob_df, x=:Prob, kind = "histogram", nbinsx=100, 
Layout(yaxis_type = "log"))

data: [
  "histogram with fields nbinsx, type, x, xaxis, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

In [62]:
names(all_live_chart_data)[41:end]

15-element Vector{String}:
 "sn"
 "unc_sn"
 "sp"
 "unc_sp"
 "binding"
 "unc_ba"
 "atomic_mass"
 "unc_am"
 "massexcess"
 "unc_me"
 "ENSDFpublicationcut-off"
 "ENSDFauthors"
 "Extraction_date"
 "Decay or Emission type"
 "nuclide"

In [63]:
all_live_chart_data[!, "Decay or Emission type"]

20202-element PooledArrays.PooledVector{String, UInt32, Vector{UInt32}}:
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 ⋮
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"

In [64]:
all_live_chart_data[!, "half_life_sec"]

20202-element Vector{String}:
 "613.9"
 "1.7547604425822527e-22"
 " "
 " "
 " "
 "388781328.00697345"
 " "
 "8.608258774931805e-23"
 "2.943469129492811e-22"
 "5.069307945237619e-21"
 " "
 " "
 "7.040705479496693e-22"
 ⋮
 " "
 "0.17"
 "0.22"
 "0.65"
 "0.0083"
 " "
 "0.0128"
 "0.053"
 " "
 "0.014"
 "0.051"
 "0.00058"

Note that based on the documentation for the livechart API, the decay type denoted "B+" also includes electron capture. 

Source: 

https://www-nds.iaea.org/relnsd/vcharthtml/api_v0_guide.html#apirad_type 

In [67]:
decay_or_emission_type_dict = Dict([
    "B-" =>  "beta minus decay", "A" =>  "alpha decay", "B+" =>  "beta plus decay",
    "EC" => "auger and conversion electron"
])
function get_half_life(nuclide, decay_type)
    if decay_type == "EC+B+" 
        decay_type = "B+"
    end
    if decay_type in keys(decay_or_emission_type_dict)
        decay_type_long = decay_or_emission_type_dict[decay_type]
        row = [row for row in 1:size(all_live_chart_data)[1]
                if ((all_live_chart_data[row, "nuclide"] ==  nuclide) && 
                (decay_type_long == all_live_chart_data[row, "Decay or Emission type"])
                )][1]
        return all_live_chart_data[row, "half_life_sec"]
    end
    if decay_type in values(decay_or_emission_type_dict)
        row = [row for row in 1:size(all_live_chart_data)[1]
                if (all_live_chart_data[row, "nuclide"] ==  nuclide && 
                decay_type == all_live_chart_data[row, "Decay or Emission type"])][1]
        return all_live_chart_data[row, "half_life_sec"]
    end
    return decay_type * " not accounted for "
end

get_half_life (generic function with 1 method)

In [68]:
all_live_chart_data[!, "nuclide"] 

20202-element Vector{String}:
 "1Nn"
 "4N"
 "6N"
 "1H"
 "2H"
 "3H"
 "4H"
 "5H"
 "6H"
 "7H"
 "3He"
 "4He"
 "5He"
 ⋮
 "287Mc"
 "288Mc"
 "289Mc"
 "290Mc"
 "290Lv"
 "291Lv"
 "292Lv"
 "293Lv"
 "294Lv"
 "293Ts"
 "294Ts"
 "294Og"

In [69]:
all_live_chart_data[!, "Decay or Emission type"]

20202-element PooledArrays.PooledVector{String, UInt32, Vector{UInt32}}:
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 ⋮
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"

In [70]:
decay_type = "B-"
nuclide = "3H"
decay_type_long = decay_or_emission_type_dict[decay_type]


"beta minus decay"

In [71]:
all_live_chart_data[6, "Decay or Emission type"]

"beta minus decay"

In [72]:
function get_daughter_nuclide(parent, decay_type)
    if decay_type == "gamma"
        return parent
    end
    mass, el = get_mass_name(parent)
    el_index = [i for i in 1:length(element_symbols)
                if element_symbols[i] == string(el)][1]
    if decay_type == "alpha" || decay_type == "A"
        return string(mass - 4) * element_symbols[el_index - 2]
    end
    if decay_type == "beta-minus" || decay_type == "B-"
        return string(mass) * element_symbols[el_index + 1]
    end
    if decay_type == "beta-plus" || decay_type == "B+" ||  decay_type == "EC+B+"
        return string(mass) * element_symbols[el_index - 1]
    end
    if decay_type == "neutron" || decay_type == "N"
        return string(mass-1) * el
    end
    if decay_type == "proton" || decay_type == "P"
        return string(mass-1) * element_symbols[el_index - 1]
    end
    if decay_type == "double neutron" || decay_type == "2N"
        return string(mass-2) * el
    end
    if decay_type == "double proton" || decay_type == "2P"
        return string(mass-2) * element_symbols[el_index - 2]
    end
    if decay_type == "double neutron" || decay_type == "2N"
        return string(mass-2) * element_symbols[el_index - 1]
    end
    return decay_type * " not yet accounted for"
end

get_daughter_nuclide (generic function with 1 method)

In [73]:
livechart_abbreviation_decay_types = Dict([
    "A" => "alpha", "B-" => "beta-minus", "B+" => "beta-plus", 
    "N" => "neutron", "P" => "proton", "EC+B+" => "beta-plus and electron capture",
    "EC" => "electron capture"
])
unique(all_live_chart_data[!,"decay_1"])

21-element Vector{String}:
 "B-"
 " "
 "N"
 "2N"
 "P"
 "A"
 "EC"
 "EC+B+"
 "2P"
 "B+P"
 "B-N"
 "B-2N"
 "B+"
 "2B-"
 "ECP+EC2P"
 "2EC"
 "IT"
 "ECP"
 "2B+"
 "SF"
 "ECSF"

The function named decay returns all vector describing all of the possible decay paths for a given parent nuclide. Each decay path in the vector includes the decay type, the probability of that decay type occurring as a percentage, and what daughter isotope will be produced. 

In [74]:
function decay(parent)
    if occursin("not yet accounted for", parent)
        return parent
    end
    row = 1
    #mass, elem = 
    while (row < size(all_live_chart_data)[1]
        && all_live_chart_data[row, "nuclide"] != parent)
        row += 1
    end 
    decay_types = (all_live_chart_data[row, "decay_1"], all_live_chart_data[row, "decay_2"],
                    all_live_chart_data[row, "decay_3"])
    if decay_types[1] == " "
        return "stable"
    end
    decay_percents =  (all_live_chart_data[row, "decay_1_%"], all_live_chart_data[row, "decay_2_%"],
                        all_live_chart_data[row, "decay_3_%"])
    direct_daughters = [get_daughter_nuclide(parent, decay_type) 
                        for decay_type in decay_types if decay_type != " "]
    decay_energies = Vector{Any}(undef, length(direct_daughters))
    half_lives = Vector{Any}(undef, length(direct_daughters))
    for i in 1:length(direct_daughters)
        decay_type = decay_types[i] 
        half_lives[i] = get_half_life(parent, decay_type)
        if !occursin("not accounted for ", half_lives[i])
            half_lives[i] = half_lives[i] * "seconds"
        end
        if decay_type in keys(livechart_abbreviation_decay_types)
            decay_energies[i] = string(find_avg_decay_energy(parent, 
                        livechart_abbreviation_decay_types[decay_type])) * "keV"
        else 
            decay_energies[i] = decay_type * " not yet accounted for"
        end
    end
    return [(decay_types[i], decay_percents[i] * "%", decay_energies[i],
            half_lives[i], direct_daughters[i])
            for i in 1:length(direct_daughters)]
end
chain = decay("238U")

2-element Vector{NTuple{5, String}}:
 ("A", "100%", "4187.13keV", "1.4099634525447706e17seconds", "234Th")
 ("SF", "0.0000545%", "SF not yet accounted for", "SF not accounted for ", "SF not yet accounted for")

Each decay route will be stored as its own string of the form isotope,decay type,probability,average energy released, daughter isotope. 

For example,

"3H,B-,100%,5.69keV,3He"

 means that 100% of tritium decays are beta-minus decay, and on average the produced beta particles will have a kinetic energy of 5.69 keV.

In [75]:
function add_to_decay_route_string(parent)
    if occursin(",", parent) 
        if decay_route_complete(parent)
            return parent
        end
        decay_route = parent
        parent = split(parent, ",")[end]
        decay_result = decay(parent)
        if typeof(decay_result) == String
            return [decay_route * "," * decay_result]
        end
        return [decay_route * "," * join(d, ",") for d in decay(parent)]
    end
    decay_result = decay(parent)
    if typeof(decay_result) == String
        return [parent * "," * decay_result]
    end
    return [parent * "," * join(d, ",") for d in decay(parent)]
end

add_to_decay_route_string (generic function with 1 method)

In [76]:
decay_route_complete(decay_route) = (occursin("not yet accounted for", decay_route)
                                    ||occursin("stable", decay_route))
all_decay_routes_complete(decay_routes) = (sum([
    decay_route_complete(decay_route) for decay_route in decay_routes]) 
    == length(decay_routes))

all_decay_routes_complete (generic function with 1 method)

In [77]:
decay_route_strs = [add_to_decay_route_string("238U")]
!all_decay_routes_complete(add_to_decay_route_string("238U"))

true

In [78]:
function make_complete_decay_chain(nuclide)
    decay_route_strs = add_to_decay_route_string(nuclide)
    while !all_decay_routes_complete(decay_route_strs)
        decay_routes_copy = decay_route_strs 
        for i in 1:length(decay_routes_copy)
            if !decay_route_complete(decay_route_strs[i])
                new_decay_routes = add_to_decay_route_string(decay_route_strs[i])
                decay_route_strs[i] = string(new_decay_routes[1])
                if length(new_decay_routes) > 1
                    decay_route_strs = vcat(decay_route_strs, new_decay_routes[2:end])
                end
            end
        end
    end
    return decay_route_strs
end

make_complete_decay_chain (generic function with 1 method)

In [79]:
make_complete_decay_chain("238U")

25-element Vector{String}:
 "238U,A,100%,4187.13keV,1.409963" ⋯ 551 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,SF,0.0000545%,SF not yet a" ⋯ 28 bytes ⋯ "d for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 160 bytes ⋯ " for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 161 bytes ⋯ " for ,Mg not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 218 bytes ⋯ "or ,24NE not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 210 bytes ⋯ " for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 261 bytes ⋯ "for ,14C not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 549 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 546 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 540 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 419 bytes ⋯ "for ,B-A not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 538 bytes ⋯ "11955686.4seconds,206

For small nuclides, it only takes about 1 second for 100 nuclides. For large nuclides, it takes 1 minute for 100 nuclides. 

Export the comprehensive decay chains of each nuclide into a directory and store them in the Julia workspace as a dictionary. 

In [80]:
comprehensive_decay_chains_dir = cross_section_dir * "Comprehensive_decay_chains\\"
decay_dict = Dict([])
for nuclide in livechart_nuclides
    try
        decay_dict[nuclide] = make_complete_decay_chain(nuclide)
        CSV.write(comprehensive_decay_chains_dir * nuclide * ".csv", 
                    DataFrame(decay_routes = decay_dict[nuclide]))
    catch
        println(nuclide, " did not work")
    end
end
decay_dict

1Nn did not work
5He did not work
9He did not work
10He did not work
4Li did not work
5Li did not work
10Li did not work
12Li did not work
6Be did not work
7Be did not work
13Be did not work
15Be did not work
16Be did not work
7B did not work
9B did not work
16B did not work
18B did not work
20B did not work
10N did not work
11N did not work
24N did not work
15F did not work
28F did not work
30F did not work
33Ne did not work
18Na did not work
19Na did not work
20Na did not work
36Na did not work
20Mg did not work
39Mg did not work
21Al did not work
43Al did not work
25P did not work
26S did not work
29Cl did not work
30Cl did not work
52Cl did not work
31Ar did not work
37Ar did not work
33K did not work
34K did not work
56K did not work
34Ca did not work
41Ca did not work
58Ca did not work
38Sc did not work
39Sc did not work
61Sc did not work
44Ti did not work
64Ti did not work
42V did not work
43V did not work
49V did not work
67V did not work
45Cr did not work
51Cr did not work
44M

Dict{Any, Any} with 2679 entries:
  "145Te" => ["145Te,B-,100%,2827.62keV, seconds,145I,B-, %,2468.0keV,0.0000004…
  "43Si"  => ["43Si,B-N, %,B-N not yet accounted for,B-N not accounted for ,B-N…
  "113Sn" => ["113Sn,EC+B+,100%,0.0keV,9943776seconds,EC+B+ not yet accounted f…
  "106Nb" => ["106Nb,B-,100%,3313.0keV,1.02seconds,106Mo,B-,100%,1212.18keV,8.7…
  "230Pu" => ["230Pu,A,100%,7045.98keV,102seconds,226U,A,100%,7539.15keV,0.268s…
  "168Eu" => ["168Eu,B-,100%,2168.0keV,0.2seconds,168Gd,B-,100%,1521.0keV,3.03s…
  "160Dy" => ["160Dy,stable"]
  "18Ne"  => ["18Ne,EC+B+,100%,1499.16keV,1.672seconds,EC+B+ not yet accounted …
  "75Cu"  => ["75Cu,B-,100%,2688.0keV,1.224seconds,75Zn,B-,100%,1893.42keV,10.2…
  "162Sm" => ["162Sm,B-,100%,1383.0keV,2.4seconds,162Eu,B-,100%,655.68keV,10.6s…
  "57V"   => ["57V,B-,100%,3558.89keV,0.32seconds,57Cr,B-,100%,1948.93keV,21.1s…
  "208Po" => ["208Po,A,99.996%,5114.7keV,91451971.47436762seconds,204Pb,A, %,19…
  "167Re" => ["167Re,EC+B+,99%,0.0keV,5.9seco

In [81]:
decay_dict["207Bi"]

1-element Vector{String}:
 "207Bi,EC+B+,100%,0.146keV,995621014.4983776seconds,EC+B+ not yet accounted for"